**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Check the params of the kube-apiserver

Add `PodSecurityPolicy` if necessary. 
The server will automatically restart on saving the file

In [1]:
grep -E '$|PodSecurityPolicy|RBAC' --color=always /etc/kubernetes/manifests/kube-apiserver.yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubeadm.kubernetes.io/kube-apiserver.advertise-address.endpoint: 192.168.49.2:8443
  creationTimestamp: null
  labels:
    component: kube-apiserver
    tier: control-plane
  name: kube-apiserver
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-apiserver
    - --advertise-address=192.168.49.2
    - --allow-privileged=true
    - --authorization-mode=Node,RBAC
    - --client-ca-file=/var/lib/minikube/certs/ca.crt
    - --enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,MutatingAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota
    - --enable-bootstrap-token-auth=true
    - --etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
    - --etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
    - --etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
    - --etcd-servers=https://127.0.0.1:2379
    - --kubelet-clien

Verify the process is running with appropriate parameters

In [2]:
kubectl apply -f https://k8s.io/examples/application/deployment.yaml

deployment.apps/nginx-deployment created


In [4]:
kubectl get all

NAME                                    READY   STATUS    RESTARTS   AGE
pod/nginx-deployment-66b6c48dd5-bmrp2   1/1     Running   0          17s
pod/nginx-deployment-66b6c48dd5-ssm2v   1/1     Running   0          17s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   106s

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-deployment   2/2     2            2           17s

NAME                                          DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-deployment-66b6c48dd5   2         2         2       17s


## Everything works, we are fine
### let's destroy the shiny, happyworld

In [5]:
kubectl delete deployment nginx-deployment

deployment.apps "nginx-deployment" deleted


In [6]:
sed -i 's/--enable-admission-plugins=.*$/&,PodSecurityPolicy/' /etc/kubernetes/manifests/kube-apiserver.yaml

In [7]:
grep -E '$|PodSecurityPolicy|RBAC' --color=always /etc/kubernetes/manifests/kube-apiserver.yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubeadm.kubernetes.io/kube-apiserver.advertise-address.endpoint: 192.168.49.2:8443
  creationTimestamp: null
  labels:
    component: kube-apiserver
    tier: control-plane
  name: kube-apiserver
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-apiserver
    - --advertise-address=192.168.49.2
    - --allow-privileged=true
    - --authorization-mode=Node,RBAC
    - --client-ca-file=/var/lib/minikube/certs/ca.crt
    - --enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,MutatingAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota,PodSecurityPolicy
    - --enable-bootstrap-token-auth=true
    - --etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
    - --etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
    - --etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
    - --etcd-servers=https://127.0.0.1:2379
   

In [8]:
ps auxwww | grep  -E "kube-apiserver|PodSecurityPolicy" --color=always |  sed  "s/ --/\n--/g"  

root        4032 21.8  2.2 1104196 347200 ?      Ssl  14:19   0:08 kube-apiserver
--advertise-address=192.168.49.2
--allow-privileged=true
--authorization-mode=Node,RBAC
--client-ca-file=/var/lib/minikube/certs/ca.crt
--enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,MutatingAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota,PodSecurityPolicy
--enable-bootstrap-token-auth=true
--etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
--etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
--etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
--etcd-servers=https://127.0.0.1:2379
--kubelet-client-certificate=/var/lib/minikube/certs/apiserver-kubelet-client.crt
--kubelet-client-key=/var/lib/minikube/certs/apiserver-kubelet-client.key
--kubelet-preferred-address-types=InternalIP,ExternalIP,Hostname
--proxy-client-cert-file=/var/lib/minikube/certs/front-proxy-client.crt
--proxy-client-ke

## Check the influence of empty PodSecurityPolicies

Can we start an nginx?

In [9]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: default
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
EOF

deployment.apps/nginx-deployment created


In [11]:
kubectl get all

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2m50s

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-deployment   0/1     0            0           8s

NAME                                          DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-deployment-6799fc88d8   1         0         0       8s


In [28]:
RS=$(kubectl get rs | awk '/nginx-deployment/{print $1}' | head -1)
echo $RS

nginx-deployment-6799fc88d8


In [29]:
kubectl describe rs $RS | grep --color=always -E '$|FailedCreate|forbidden' 

Name:           nginx-deployment-6799fc88d8
Namespace:      default
Selector:       app=nginx,pod-template-hash=6799fc88d8
Labels:         app=nginx
                pod-template-hash=6799fc88d8
Annotations:    deployment.kubernetes.io/desired-replicas: 1
                deployment.kubernetes.io/max-replicas: 2
                deployment.kubernetes.io/revision: 1
Controlled By:  Deployment/nginx-deployment
Replicas:       0 current / 1 desired
Pods Status:    0 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  app=nginx
           pod-template-hash=6799fc88d8
  Containers:
   nginx:
    Image:        nginx
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type             Status  Reason
  ----             ------  ------
  ReplicaFailure   True    FailedCreate
Events:
  Type     Reason        Age                From                   Message
  ----     ------        ----          

Without an existing PSP nobody can start anything. 

Note that the *replicaset* is reporting the failure.

The pod does not exist, thus there are no logs we can watch

In [14]:
kubectl get psp

No resources found


# Policies

* can be applied to each account, including 
 * Users
 * ServiceAccounts
* give fine grained control over rights

We start with the *restrictive* Policy from the Kubernets Documentation
https://kubernetes.io/docs/concepts/policy/pod-security-policy


In [15]:
kubectl create -f  - <<EOF
apiVersion: policy/v1beta1
kind: PodSecurityPolicy
metadata:
  name: restrictive
spec:
  privileged: false
  hostNetwork: false
  allowPrivilegeEscalation: false
  defaultAllowPrivilegeEscalation: false
  hostPID: false
  hostIPC: false
  runAsUser:
    rule: RunAsAny
  fsGroup:
    rule: RunAsAny
  seLinux:
    rule: RunAsAny
  supplementalGroups:
    rule: RunAsAny
  volumes:
  - 'configMap'
  - 'downwardAPI'
  - 'emptyDir'
  - 'persistentVolumeClaim'
  - 'secret'
  - 'projected'
  allowedCapabilities:
  - '*'
EOF

podsecuritypolicy.policy/restrictive created


In [16]:
kubectl delete deployment nginx-deployment

deployment.apps "nginx-deployment" deleted


In [17]:
kubectl create -f- <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: pause
spec:
  containers:
    - name: pause
      image: k8s.gcr.io/pause
EOF

pod/pause created


In [20]:
kubectl get pods

NAME    READY   STATUS    RESTARTS   AGE
pause   1/1     Running   0          7s


In [21]:
kubectl get pod $(kubectl get po | egrep -o nginx[A-Za-z0-9-]+) -o yaml | grep -i psp

      kubernetes.io/psp: restrictive


In [23]:
kubectl get pod pause -o yaml | grep --color=always -E '$|psp' 

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/psp: restrictive
  creationTimestamp: "2021-11-25T14:22:19Z"
  name: pause
  namespace: default
  resourceVersion: "744"
  uid: 36ad33b9-aa1d-4348-8fb9-69e6d72153a2
spec:
  containers:
  - image: k8s.gcr.io/pause
    imagePullPolicy: Always
    name: pause
    resources: {}
    securityContext:
      allowPrivilegeEscalation: false
    terminationMessagePath: /dev/termination-log
    terminationMessagePolicy: File
    volumeMounts:
    - mountPath: /var/run/secrets/kubernetes.io/serviceaccount
      name: kube-api-access-mcggx
      readOnly: true
  dnsPolicy: ClusterFirst
  enableServiceLinks: true
  nodeName: minikube
  preemptionPolicy: PreemptLowerPriority
  priority: 0
  restartPolicy: Always
  schedulerName: default-scheduler
  securityContext: {}
  serviceAccount: default
  serviceAccountName: default
  terminationGracePeriodSeconds: 30
  tolerations:
  - effect: NoExecute
    key: node.kubernetes.io/not-ready
 

## Again


Go back to the nginx example and start again
Let us restart the deployment again

Still not working because the psp is not bound

## Roles 

We create a restrictive *ClusterRole*

In [24]:
kubectl apply -f - <<EOF
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: psp-restrictive
rules:
- apiGroups:
  - extensions
  resources:
  - podsecuritypolicies
  resourceNames:
  - restrictive
  verbs:
  - use
EOF

clusterrole.rbac.authorization.k8s.io/psp-restrictive created


## RoleBindings

We need to bind the role to a user

In [23]:
kubectl create -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: restrictive
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: psp-restrictive
subjects:
- kind: ServiceAccount
  name: default
  namespace: default
EOF

rolebinding.rbac.authorization.k8s.io/restrictive created


In [41]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: default
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
EOF

deployment.apps/nginx-deployment created


In [44]:
kubectl get all

NAME                                    READY   STATUS    RESTARTS   AGE
pod/nginx-deployment-6799fc88d8-2znzz   1/1     Running   0          24s
pod/pause                               1/1     Running   0          12m

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   17m

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-deployment   1/1     1            1           24s
deployment.apps/nginx-strict       0/3     0            0           8m30s

NAME                                          DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-deployment-6799fc88d8   1         1         1       24s
replicaset.apps/nginx-strict-746cddb8f        3         0         0       8m30s


## Create a Permissive Policy

In [66]:
kubectl apply -f - <<EOF
apiVersion: policy/v1beta1
kind: PodSecurityPolicy
metadata:
  name: privileged
spec:
  privileged: true
  allowPrivilegeEscalation: true
  allowedCapabilities:
  - '*'
  volumes:
  - '*'
  hostNetwork: true
  hostPorts:
  - min: 0
    max: 65535
  hostIPC: true
  hostPID: true
  runAsUser:
    rule: 'RunAsAny'
  seLinux:
    rule: 'RunAsAny'
  supplementalGroups:
    rule: 'RunAsAny'
  fsGroup:
    rule: 'RunAsAny'
EOF

podsecuritypolicy.policy/privileged created


In [39]:
kubectl apply -f - <<EOF
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: psp-privileged
rules:
- apiGroups:
  - extensions
  resources:
  - podsecuritypolicies
  resourceNames:
  - restrictive
  verbs:
  - use
EOF

clusterrole.rbac.authorization.k8s.io/psp-privileged created


In [40]:
kubectl create -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name:  privileged
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: psp-privileged
subjects:
- kind: ServiceAccount
  name: default
  namespace: default
EOF

rolebinding.rbac.authorization.k8s.io/privileged created


## Which policy has been applied

how can we detect, which policy is in use for a given pod 

In [45]:
POD=$(kubectl get  pod | awk '/nginx-deployment/{print $1}' | head -1)
echo $POD

nginx-deployment-6799fc88d8-2znzz


In [46]:
kubectl get pods

NAME                                READY   STATUS    RESTARTS   AGE
nginx-deployment-6799fc88d8-2znzz   1/1     Running   0          64s
pause                               1/1     Running   0          13m


Pedestrians approach

In [61]:
kubectl get pod $(kubectl get po | egrep -o nginx[A-Za-z0-9-]+) -o yaml | grep -i psp

      kubernetes.io/psp: restrictive


Enjoy the beauty of the jsonpath template,

`.metadata.annotations` is a map

In [48]:
kubectl get pod $POD -o jsonpath='{.metadata.annotations}'

{"kubernetes.io/psp":"restrictive"}

`kubernetes\.io/psp` is a valid name
* the `/` is a usual character in a string
* the `.` needs to be escaped

In [49]:
kubectl get pod $POD -o jsonpath='{.metadata.annotations.kubernetes\.io/psp}'

restrictive

and now all pods

In [50]:
kubectl get pod --all-namespaces -o jsonpath='{.items[*].metadata.annotations.kubernetes\.io/psp}'

restrictive restrictive

In [60]:
kubectl get psp

NAME         PRIV   CAPS   SELINUX    RUNASUSER   FSGROUP    SUPGROUP   READONLYROOTFS   VOLUMES
privileged   true   *      RunAsAny   RunAsAny    RunAsAny   RunAsAny   false            *


In [52]:
kubectl delete psp restrictive

podsecuritypolicy.policy "restrictive" deleted


In [67]:
kubectl delete deployment nginx-deployment

Error from server (NotFound): deployments.apps "nginx-deployment" not found


: 1

In [80]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-strict
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      securityContext:
        runAsUser: 110
        runAsGroup: 110
        fsGroup: 110
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF


deployment.apps/nginx-strict created


In [82]:
kubectl get pods

NAME                            READY   STATUS              RESTARTS   AGE
nginx-strict-74f7c96556-chdhb   1/1     Running             0          7s
nginx-strict-74f7c96556-g2g8s   1/1     Running             0          7s
nginx-strict-74f7c96556-qvb26   0/1     ContainerCreating   0          7s
pause                           1/1     Running             0          28m


In [ ]:
kubectl delete deployment nginx-strict

In [83]:
kubectl get all

NAME                                READY   STATUS    RESTARTS   AGE
pod/nginx-strict-74f7c96556-chdhb   1/1     Running   0          23s
pod/nginx-strict-74f7c96556-g2g8s   1/1     Running   0          23s
pod/nginx-strict-74f7c96556-qvb26   1/1     Running   0          23s
pod/pause                           1/1     Running   0          29m

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   34m

NAME                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-strict   3/3     3            3           23s

NAME                                      DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-strict-74f7c96556   3         3         3       23s


In [84]:
kubectl describe rs  nginx-strict-74f7c96556

Name:           nginx-strict-74f7c96556
Namespace:      default
Selector:       app=nginx,pod-template-hash=74f7c96556
Labels:         app=nginx
                pod-template-hash=74f7c96556
Annotations:    deployment.kubernetes.io/desired-replicas: 3
                deployment.kubernetes.io/max-replicas: 4
                deployment.kubernetes.io/revision: 1
Controlled By:  Deployment/nginx-strict
Replicas:       3 current / 3 desired
Pods Status:    3 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  app=nginx
           pod-template-hash=74f7c96556
  Containers:
   nginx:
    Image:        nginxinc/nginx-unprivileged
    Port:         8080/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Events:
  Type    Reason            Age   From                   Message
  ----    ------            ----  ----                   -------
  Normal  SuccessfulCreate  30s   replicaset-controller  Created pod: nginx-strict-74f7c9

## Summary 

* we have started one pod
* with the restrictive PodSecurityPolicy
* bound to the default serviceaccount
* in the default namespace

[ ![High Five](https://image.freepik.com/free-photo/blonde-african-young-women-giving-high-five-against-grey-background_23-2148178786.jpg) ](https://www.freepik.com/free-photo/blonde-african-young-women-giving-high-five-against-grey-background_4732025.htm)

Attribution: [Background photo created by freepik](https://www.freepik.com/photos/background), thanks to [freepik](https://www.freepik.com)